In [0]:
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="independente")

In [0]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Species", outputCol="label")

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="label")

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vector, indexer, mlp])

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=MulticlassClassificationEvaluator(),numFolds=3)  

In [0]:
modelo = crossval.fit(irisTreino)

In [0]:
previsao = modelo.transform(irisTeste)
previsao.show()

+---+-------------+------------+-------------+------------+---------------+-----------------+-----+--------------------+--------------------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|        Species|     independente|label|       rawPrediction|         probability|prediction|
+---+-------------+------------+-------------+------------+---------------+-----------------+-----+--------------------+--------------------+----------+
|  3|          4.7|         3.2|          1.3|         0.2|    Iris-setosa|[4.7,3.2,1.3,0.2]|  0.0|[89.2222743954405...|[1.0,9.4490165666...|       0.0|
|  5|          5.0|         3.6|          1.4|         0.2|    Iris-setosa|[5.0,3.6,1.4,0.2]|  0.0|[89.2480910142160...|[1.0,9.0988126910...|       0.0|
| 11|          5.4|         3.7|          1.5|         0.2|    Iris-setosa|[5.4,3.7,1.5,0.2]|  0.0|[89.2504523921351...|[1.0,9.0674357625...|       0.0|
| 13|          4.8|         3.0|          1.4|         0.1|    Iris-setosa|[4.8,3.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9090909090909091
